In [1]:
import pandas as pd
import numpy as np

# Load the raw metrics file

In [2]:
filename = "../data/all_results_9q-square_raw.csv"
df = pd.read_csv(filename)

In [3]:
df.head()

,id,architecture,mode,index,population,n_iterations,crossover,mutation,n_cnots,time,destination_file
0,../circuits/steiner/9qubits/3/Original0.qasm,9q-square,steiner,NaN,NaN,NaN,NaN,NaN,10,0.006540,../circuits/steiner/9qubits/3/9q-square/steine...
1,../circuits/steiner/9qubits/3/Original1.qasm,9q-square,steiner,NaN,NaN,NaN,NaN,NaN,15,0.010701,../circuits/steiner/9qubits/3/9q-square/steine...
2,../circuits/steiner/9qubits/3/Original10.qasm,9q-square,steiner,NaN,NaN,NaN,NaN,NaN,27,0.010492,../circuits/steiner/9qubits/3/9q-square/steine...
3,../circuits/steiner/9qubits/3/Original11.qasm,9q-square,steiner,NaN,NaN,NaN,NaN,NaN,27,0.011411,../circuits/steiner/9qubits/3/9q-square/steine...
4,../circuits/steiner/9qubits/3/Original12.qasm,9q-square,steiner,NaN,NaN,NaN,NaN,NaN,14,0.009671,../circuits/steiner/9qubits/3/9q-square/steine...


# Preprocessing
Specify which parameters to use.
You can select multiple in a list to compare them with each other.

In [4]:
architectures = ["9q-square"]
modes = ["steiner", "genetic_steiner", "quilc"]
populations = [30]
iterations = [15]
crossovers = [0.8]
mutations = [0.2]

Remove all rows from the csv that do not conform to the specified parameters.

In [5]:
df2 = df[df["architecture"].isin(architectures)]
df2 = df2[df2["mode"].isin(modes)]

if "genetic_steiner" in modes or "genetic_gauss" in modes:
    df2 = df2[df2["population"].isnull() | df2["population"].isin(populations)]
    df2 = df2[df2["n_iterations"].isnull() | df2["n_iterations"].isin(iterations)]
    df2 = df2[df2["crossover"].isnull() | df2["crossover"].isin(crossovers)]
    df2 = df2[df2["mutation"].isnull() | df2["mutation"].isin(mutations)]

Add depth information from the folder structure in the id filename.

In [6]:
df2["depth"] = df2["id"].apply(lambda x: int(x.split("/")[4]))

# Aggregate the data

Fill up the NaN values in the table with -1 for grouping

In [7]:
df2 = df2.fillna(-1)

Aggregate the multiple compilations from quilc for each file by picking the circuit with the lowest n_cnots score.

In [8]:
quil_group = df2.groupby(["id", "architecture", "mode", "population", "n_iterations", "crossover", "mutation"])
df3 = df2.loc[quil_group.idxmin()["n_cnots"]].drop("index", axis=1)

Calculate the mean and median metrics per combination of mode, depth

In [9]:
# Calculate the mean and median gate count
grouped = df3.groupby(["mode", "depth", "population", "n_iterations", "crossover", "mutation"])
means = grouped.mean()
medians = grouped.median().reindex(means.index)

Combine the results in an easily readable format

In [10]:
# Combine the mean gate counts to a single table.
depths = means.index.levels[1].values
modes = means.index.levels[0].values
indices = [i for i in range(len(means.index.levels)) if means.index.levels[i].name != "depth"]
names = [means.index.levels[i].name for i in indices]
metrics = ["n_cnots", "time"]
combined_df = None
for i in depths:
    query = "depth == " + str(i)
    mean_query_results = means.query(query)
    median_query_results = medians.query(query)
    result = {mean_query_results.index.levels[i].name:[v[i] for v in mean_query_results.index.values] for i in indices}
    for metric in metrics:
        result[query + ":mean:" + metric] = mean_query_results[metric].values
        result[query + ":median:" + metric] = median_query_results[metric].values
    result_df = pd.DataFrame(result).set_index(names)
    result_df.columns = pd.MultiIndex.from_tuples([tuple(c.split(":")) for c in result_df.columns])
    if combined_df is None:
        combined_df = result_df
    else:
        combined_df = combined_df.join(result_df)

# Visualize the table

Define how to color the table

In [11]:
def highlight_cnots(s):
    '''
    highlight the best cnot counts in a Series.
    '''
    if s.name[-1] == "n_cnots":
        depth = int(s.name[0].split(" ")[-1])
        def color_func(v):
            if v <= depth:
                return "green"
            elif v <= 1.5*depth:
                return "cyan"
            elif v <= 2*depth:
                return "yellow"
            else:
                return "white"
        return ["background-color: " + color_func(v) for v in s]
    else:
        return ['' for v in s]

Show the colored table

In [12]:
combined_df.style.apply(highlight_cnots)

# Store the results
Specify the filename:

In [13]:
destination_file = "../data/9q-square_results.csv"

And store the dataframe to csv with practical column names.

In [14]:
temp_columns = combined_df.columns
combined_df.columns = ["_".join(c)[len("depth == "):] for c in zip(*[combined_df.columns.get_level_values(i) for i in range(3)])]
combined_df.to_csv(destination_file)
combined_df.columns = temp_columns